<img src="img/header.png" />

# Aula 04 - Machine Learning

Já pensou uma bola de cristal pra prever os salários dos profissionais de dados?? Vamos fazer um modelinho com scikit learn na nossa base do challenge pra criar uma calculadora de salários!

Mas antes do Streamlit, vamos treinar nosso modelinho lindo!

In [5]:
"""
Modelo de Machine Learning para Predição de Salários (REGRESSÃO)
Aula 04 - Machine Learning Básico

Este script treina um modelo supervisionado de REGRESSÃO para predizer salários
com base em características dos profissionais de dados.
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pickle
import warnings
warnings.filterwarnings('ignore')

In [1]:
def converter_faixa_salarial_para_valor(faixa):
    """Converte faixa salarial para valor numérico (média da faixa)"""
    mapeamento = {
        'Menos de R$ 1.000/mês': 800,  # Assumindo média de 800
        'de R$ 1.001/mês a R$ 2.000/mês': 1500,
        'de R$ 2.001/mês a R$ 3.000/mês': 2500,
        'de R$ 3.001/mês a R$ 4.000/mês': 3500,
        'de R$ 4.001/mês a R$ 6.000/mês': 5000,
        'de R$ 6.001/mês a R$ 8.000/mês': 7000,
        'de R$ 8.001/mês a R$ 12.000/mês': 10000,
        'de R$ 12.001/mês a R$ 16.000/mês': 14000,
        'de R$ 16.001/mês a R$ 20.000/mês': 18000,
        'de R$ 20.001/mês a R$ 25.000/mês': 22500,
        'de R$ 25.001/mês a R$ 30.000/mês': 27500,
        'de R$ 30.001/mês a R$ 40.000/mês': 35000,
        'Acima de R$ 40.001/mês': 50000  # Assumindo média de 50k para a faixa aberta
    }
    return mapeamento.get(faixa, None)

In [2]:
def preprocessar_dados(df):
    """Preprocessa os dados para o modelo"""
    
    # Selecionar apenas as colunas necessárias e remover linhas com valores faltantes críticos
    colunas_modelo = ['genero', 'etnia', 'idade', 'nivel_ensino', 'area_formacao', 
                      'situacao_trabalho', 'cargo_atual', 'tempo_experiencia_dados', 
                      'uf_residencia', 'faixa_salarial']
    
    df_modelo = df[colunas_modelo].copy()
    
    # Remover linhas com cargo_atual faltante (variável muito importante)
    df_modelo = df_modelo.dropna(subset=['cargo_atual'])
    
    # Preencher valores faltantes
    df_modelo['area_formacao'] = df_modelo['area_formacao'].fillna('Outra opção')
    df_modelo['uf_residencia'] = df_modelo['uf_residencia'].fillna('SP')  # Usar SP como padrão, CUIDADO, o melhor seria usar a distribuição de todos os estados, mas aqui é só pra gente mostrar
    
    # Converter faixa salarial para valores numéricos
    df_modelo['salario_valor'] = df_modelo['faixa_salarial'].apply(converter_faixa_salarial_para_valor)
    
    # Remover registros com faixa salarial inválida
    df_modelo = df_modelo[df_modelo['salario_valor'].notna()]
    
    return df_modelo

In [3]:
def treinar_modelo():
    """Treina o modelo de machine learning (REGRESSÃO)"""
    
    print("=== TREINAMENTO DO MODELO DE PREDIÇÃO DE SALÁRIOS (REGRESSÃO) ===")
    print("Carregando dados...")
    
    # Carregar dados
    df = pd.read_csv('data/processed/dataset_salarios_dados.csv')
    print(f"Dataset original: {df.shape[0]} registros")
    
    # Preprocessar dados
    df_modelo = preprocessar_dados(df)
    print(f"Dataset após preprocessamento: {df_modelo.shape[0]} registros")
    
    # Separar features e target
    features = ['genero', 'etnia', 'idade', 'nivel_ensino', 'area_formacao', 
                'situacao_trabalho', 'cargo_atual', 'tempo_experiencia_dados', 'uf_residencia']

    # Separando em features e target
    X = df_modelo[features].copy()
    y = df_modelo['salario_valor']  # Agora é valor numérico
    
    print(f"Features utilizadas: {features}")
    print(f"Estatísticas do salário (target):")
    print(f"Média: R$ {y.mean():,.2f}")
    print(f"Mediana: R$ {y.median():,.2f}")
    print(f"Mínimo: R$ {y.min():,.2f}")
    print(f"Máximo: R$ {y.max():,.2f}")
    print(f"Desvio padrão: R$ {y.std():,.2f}")
    
    # Encoding das variáveis categóricas (transformando categorias em números, pois o modelo é BURRIM e só entende números)
    label_encoders = {}
    for coluna in features:
        if X[coluna].dtype == 'object':
            le = LabelEncoder()
            X[coluna] = le.fit_transform(X[coluna].astype(str))
            label_encoders[coluna] = le
    
    # Dividir em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"\nDivisão dos dados:")
    print(f"Treino: {X_train.shape[0]} registros")
    print(f"Teste: {X_test.shape[0]} registros")
    
    # Treinar modelo Random Forest Regressor
    print("\nTreinando modelo Random Forest Regressor...")
    modelo = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42
    )

    # Treinando o modelo, ou seja, o modelo está aprendendo com a nossa base de dados!
    modelo.fit(X_train, y_train)
    
    # Avaliar modelo
    print("\n=== AVALIAÇÃO DO MODELO ===")
    y_pred = modelo.predict(X_test)
    
    # Métricas de regressão
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"R² Score: {r2:.3f}")
    print(f"RMSE: R$ {rmse:,.2f}")
    print(f"MAE: R$ {mae:,.2f}")
    
    # Mostrar algumas predições de exemplo
    print(f"\nExemplos de predições:")
    for i in range(min(5, len(y_test))):
        real = y_test.iloc[i]
        pred = y_pred[i]
        diff = abs(real - pred)
        print(f"Real: R$ {real:,.2f} | Predito: R$ {pred:,.2f} | Diferença: R$ {diff:,.2f}")
    
    # Importância das features
    print("\nImportância das features:")
    importance_df = pd.DataFrame({
        'feature': features,
        'importance': modelo.feature_importances_
    }).sort_values('importance', ascending=False)
    
    for _, row in importance_df.iterrows():
        print(f"{row['feature']}: {row['importance']:.3f}")
    
    # Salvar modelo e encoders
    print("\n=== SALVANDO MODELO ===")
    
    # Criar dicionário com tudo que precisamos para fazer predições
    modelo_completo = {
        'modelo': modelo,
        'label_encoders': label_encoders,
        'features': features,
        'tipo': 'regressao',
        'metricas': {
            'r2': r2,
            'rmse': rmse,
            'mae': mae
        }
    }
    
    # Salvar modelo
    with open('modelo_salarios.pkl', 'wb') as f:
        pickle.dump(modelo_completo, f)
    
    print("Modelo salvo como 'modelo_salarios.pkl'")
    print("O modelo está pronto para ser usado na aplicação Streamlit!")
    
    return modelo_completo

In [6]:
modelo = treinar_modelo() 

=== TREINAMENTO DO MODELO DE PREDIÇÃO DE SALÁRIOS (REGRESSÃO) ===
Carregando dados...
Dataset original: 4863 registros
Dataset após preprocessamento: 3818 registros
Features utilizadas: ['genero', 'etnia', 'idade', 'nivel_ensino', 'area_formacao', 'situacao_trabalho', 'cargo_atual', 'tempo_experiencia_dados', 'uf_residencia']
Estatísticas do salário (target):
Média: R$ 9,908.46
Mediana: R$ 10,000.00
Mínimo: R$ 800.00
Máximo: R$ 50,000.00
Desvio padrão: R$ 7,294.90

Divisão dos dados:
Treino: 3054 registros
Teste: 764 registros

Treinando modelo Random Forest Regressor...

=== AVALIAÇÃO DO MODELO ===
R² Score: 0.464
RMSE: R$ 5,252.53
MAE: R$ 3,444.07

Exemplos de predições:
Real: R$ 7,000.00 | Predito: R$ 8,694.33 | Diferença: R$ 1,694.33
Real: R$ 1,500.00 | Predito: R$ 7,543.84 | Diferença: R$ 6,043.84
Real: R$ 7,000.00 | Predito: R$ 9,616.87 | Diferença: R$ 2,616.87
Real: R$ 22,500.00 | Predito: R$ 31,847.88 | Diferença: R$ 9,347.88
Real: R$ 14,000.00 | Predito: R$ 9,266.73 | Diferenç